##### Copyright 2021 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Generate music with an RNN

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/audio/music_generation"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/audio/music_generation.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/audio/music_generation.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/audio/music_generation.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows you how to generate musical notes using a simple recurrent neural network (RNN). You will train a model using a collection of piano MIDI files from the [MAESTRO dataset](https://magenta.tensorflow.org/datasets/maestro). Given a sequence of notes, your model will learn to predict the next note in the sequence. You can generate longer sequences of notes by calling the model repeatedly.

This tutorial contains complete code to parse and create MIDI files. You can learn more about how RNNs work by visiting the [Text generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation) tutorial.

## Setup

This tutorial uses the [`pretty_midi`](https://github.com/craffel/pretty-midi) library to create and parse MIDI files, and [`pyfluidsynth`](https://github.com/nwhitehead/pyfluidsynth) for generating audio playback in Colab.

In [2]:
# !source music/bin/activate

In [3]:
# !sudo apt install -y fluidsynth

In [4]:
# !pip install --upgrade pyfluidsynth

In [5]:
# !pip install pretty_midi
# !pip install pandas
# !pip install tensorflow
# !pip install seaborn
# !pip install matplotlib
# !pip install setuptools

In [ ]:
import sys, importlib.util

# 1. Which Python executable?
print("sys.executable:", sys.executable)

# 2. What does its module search path look like?
print("\nsys.path:")
for p in sys.path:
    print(" ", p)

# 3. Does it even know about the fluidsynth package?
spec = importlib.util.find_spec("fluidsynth")
print("\nimportlib.util.find_spec('fluidsynth'):", spec)

# 4. What pip thinks is installed here
print("\n!{sys.executable} -m pip show pyfluidsynth fluidsynth")


In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import collections
import datetime
import fluidsynth
import pretty_midi

import glob
import numpy as np
import pathlib
import pandas as pd

import seaborn as sns
import tensorflow as tf

from IPython import display
from matplotlib import pyplot as plt
from typing import Optional
import logging

In [8]:
tf.debugging.set_log_device_placement(False)
tf.get_logger().setLevel(logging.ERROR)

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Sampling rate for audio playback
_SAMPLING_RATE = 16000

## Download the Maestro dataset

In [9]:
data_dir = pathlib.Path('data/maestro-v2.0.0')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'maestro-v2.0.0-midi.zip',
      origin='https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )

The dataset contains about 1,200 MIDI files.

In [ ]:
data_dir = pathlib.Path('data/maestro-v2_extracted/maestro-v2.0.0/')
filenames = list(data_dir.rglob("*.mid*"))
print('Number of files:', len(filenames))

## Process a MIDI file

First, use ```pretty_midi``` to parse a single MIDI file and inspect the format of the notes. If you would like to download the MIDI file below to play on your computer, you can do so in colab by writing ```files.download(sample_file)```.


In [ ]:
sample_file = filenames[1]
print(sample_file)

Generate a `PrettyMIDI` object for the sample MIDI file.

In [12]:
pm = pretty_midi.PrettyMIDI(str(sample_file))

Play the sample file. The playback widget may take several seconds to load.

In [13]:
def display_audio(pm: pretty_midi.PrettyMIDI, seconds=30):
  waveform = pm.fluidsynth(fs=_SAMPLING_RATE)
  # Take a sample of the generated waveform to mitigate kernel resets
  waveform_short = waveform[:seconds*_SAMPLING_RATE]
  return display.Audio(waveform_short, rate=_SAMPLING_RATE)

In [ ]:
import sys
print(sys.executable)
print(sys.path[:3])

In [ ]:
%env HOMEBREW_PREFIX=/opt/homebrew

display_audio(pm)

Do some inspection on the MIDI file. What kinds of instruments are used?

In [ ]:
print('Number of instruments:', len(pm.instruments))
instrument = pm.instruments[0]
instrument_name = pretty_midi.program_to_instrument_name(instrument.program)
print('Instrument name:', instrument_name)

## Extract notes

In [ ]:
for i, note in enumerate(instrument.notes[:10]):
  note_name = pretty_midi.note_number_to_name(note.pitch)
  duration = note.end - note.start
  print(f'{i}: pitch={note.pitch}, note_name={note_name},'
        f' duration={duration:.4f}')

You will use three variables to represent a note when training the model: `pitch`, `step` and `duration`. The pitch is the perceptual quality of the sound as a MIDI note number. 
The `step` is the time elapsed from the previous note or start of the track.
The `duration` is how long the note will be playing in seconds and is the difference between the note end and note start times. 


Extract the notes from the sample MIDI file.

In [18]:
def midi_to_notes(midi_file: str) -> pd.DataFrame:
  pm = pretty_midi.PrettyMIDI(midi_file)
  instrument = pm.instruments[0]
  notes = collections.defaultdict(list)

  # Sort the notes by start time
  sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
  prev_start = sorted_notes[0].start

  for note in sorted_notes:
    start = note.start
    end = note.end
    notes['pitch'].append(note.pitch)
    notes['start'].append(start)
    notes['end'].append(end)
    notes['step'].append(start - prev_start)
    notes['duration'].append(end - start)
    prev_start = start

  return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

In [ ]:
raw_notes = midi_to_notes(str(sample_file))
raw_notes.head()

It may be easier to interpret the note names rather than the pitches, so you can use the function below to convert from the numeric pitch values to note names. 
The note name shows the type of note, accidental and octave number
(e.g. C#4). 

In [ ]:
get_note_names = np.vectorize(pretty_midi.note_number_to_name)
sample_note_names = get_note_names(raw_notes['pitch'])
sample_note_names[:10]

To visualize the musical piece, plot the note pitch, start and end across the length of the track (i.e. piano roll). Start with the first 100 notes

In [21]:
def plot_piano_roll(notes: pd.DataFrame, count: Optional[int] = None):
  if count:
    title = f'First {count} notes'
  else:
    title = f'Whole track'
    count = len(notes['pitch'])
  plt.figure(figsize=(20, 4))
  plot_pitch = np.stack([notes['pitch'], notes['pitch']], axis=0)
  plot_start_stop = np.stack([notes['start'], notes['end']], axis=0)
  plt.plot(
      plot_start_stop[:, :count], plot_pitch[:, :count], color="b", marker=".")
  plt.xlabel('Time [s]')
  plt.ylabel('Pitch')
  _ = plt.title(title)

In [ ]:
plot_piano_roll(raw_notes, count=100)

Plot the notes for the entire track.

In [ ]:
plot_piano_roll(raw_notes)

Check the distribution of each note variable.

In [24]:
def plot_distributions(notes: pd.DataFrame, drop_percentile=2.5):
  plt.figure(figsize=[15, 5])
  plt.subplot(1, 3, 1)
  sns.histplot(notes, x="pitch", bins=20)

  plt.subplot(1, 3, 2)
  max_step = np.percentile(notes['step'], 100 - drop_percentile)
  sns.histplot(notes, x="step", bins=np.linspace(0, max_step, 21))
  
  plt.subplot(1, 3, 3)
  max_duration = np.percentile(notes['duration'], 100 - drop_percentile)
  sns.histplot(notes, x="duration", bins=np.linspace(0, max_duration, 21))

In [ ]:
plot_distributions(raw_notes)

## Create a MIDI file

You can generate your own MIDI file from a list of notes using the function below.

In [26]:
def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str, 
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm

In [27]:
example_file = 'example.midi'
example_pm = notes_to_midi(
    raw_notes, out_file=example_file, instrument_name=instrument_name)

Play the generated MIDI file and see if there is any difference.

In [ ]:
display_audio(example_pm)

As before, you can write ```files.download(example_file)``` to download and play this file.

## Create the training dataset


Create the training dataset by extracting notes from the MIDI files. You can start by using a small number of files, and experiment later with more. This may take a couple minutes.

In [29]:
MAX_DURATION = 0.693
MAX_STEP = 0.306

In [30]:
num_files = 5
all_notes = []
for f in filenames[:num_files]:
  notes = midi_to_notes(str(f))
  all_notes.append(notes)

all_notes = pd.concat(all_notes)

## Plotting before clipping duration and step

In [ ]:
plot_distributions(all_notes, drop_percentile=0)

In [32]:
# 1. Compute bounds on your training set (e.g. 1st & 99th percentiles)
lower_s, upper_s = all_notes['step'].quantile([0.02, 0.98])
lower_d, upper_d = all_notes['duration'].quantile([0.02, 0.98])

mask = (
    (all_notes['step'].between(lower_s, upper_s)) &
    (all_notes['duration'].between(lower_d, upper_d))
)
all_notes = all_notes[mask].reset_index(drop=True)

In [ ]:
plot_distributions(all_notes, drop_percentile=0)

In [ ]:
print(all_notes["step"].max())
print(all_notes["duration"].max())

## Quantizing steps and duration

In [35]:
raw_steps = raw_notes["step"].to_numpy().astype(np.float32)
raw_durs = raw_notes["duration"].to_numpy().astype(np.float32)

In [36]:
all_steps = all_notes["step"].to_numpy().astype(np.float32)
all_durs  = all_notes["duration"].to_numpy().astype(np.float32)
N = 32
q_eps = 1e-3
def quantize(input, epsilon, num_steps):
    min_input = input.min()
    max_input = input.max()
    edges = np.geomspace(min_input + epsilon, max_input, num_steps+1)
    buckets = np.digitize(input, bins=edges) - 1
    # clamp to [0, N-1]
    buckets = np.clip(buckets, 0, N-1)
    return buckets, edges

def dequantize(buckets, edges):
    # Map each bucket back to the geometric midpoint of its interval
    lower = edges[buckets]
    upper = edges[buckets + 1]
    return np.sqrt(lower * upper)


step_buckets, step_edges = quantize(all_steps, q_eps, N)
dur_buckets, dur_edges = quantize(all_steps, q_eps, N)

all_notes["step"] = step_buckets
all_notes["duration"] = dur_buckets 

## Check that quantizing works

In [ ]:
quant_step, st_edg = quantize(raw_notes["step"], q_eps, N)
quant_dur, dr_edg = quantize(raw_notes["duration"], q_eps, N)

st_dequant = dequantize(quant_step, st_edg)
dur_dequant = dequantize(quant_dur, dr_edg)

for i, note in raw_notes.iterrows():
    print(f"duration: {note['duration']}, {dur_dequant[i]}")
    print(f"step: {note['step']}, {st_dequant[i]}")

In [ ]:
raw_notes["duration"] = dur_dequant
raw_notes["step"] = st_dequant
example_pm = notes_to_midi(
    raw_notes, out_file=example_file, instrument_name=instrument_name)

display_audio(example_pm)

In [ ]:
n_notes = len(all_notes)
print('Number of notes parsed:', n_notes)

In [ ]:
pitch_vocab = set(all_notes["pitch"])
vocab_size = len(pitch_vocab)
print('vocab size = ', vocab_size)

Next, create a `tf.data.Dataset` from the parsed notes.

In [41]:
key_order = ['pitch', 'step', 'duration']
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)

In [ ]:
train_notes.shape

In [ ]:
notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
notes_ds.element_spec

In [ ]:
plt.figure(figsize=[15, 5])
plt.hist(step_buckets, bins=N, edgecolor='black')
plt.title("Histogram of Step Buckets")
plt.xlabel("Step Bucket ID")
plt.ylabel("Frequency")
plt.show()

plt.figure(figsize=[15, 5])
plt.hist(dur_buckets, bins=N, edgecolor='black')
plt.title("Histogram of Duration Buckets")
plt.xlabel("Duration Bucket ID")
plt.ylabel("Frequency")
plt.show()

You will train the model on batches of sequences of notes. Each example will consist of a sequence of notes as the input features, and the next note as the label. In this way, the model will be trained to predict the next note in a sequence. You can find a diagram describing this process (and more details) in [Text classification with an RNN](https://www.tensorflow.org/text/tutorials/text_generation).

You can use the handy [window](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window) function with size `seq_length` to create the features and labels in this format.

In [45]:
def create_sequences(
    dataset: tf.data.Dataset, 
    seq_length: int,
    vocab_size=128,
    max_step=MAX_STEP,
    max_duration=MAX_DURATION
) -> tf.data.Dataset:
    """Returns TF Dataset of ({p_in,s_in,d_in}, {pitch,step,duration}) examples."""
    # we need seq_length inputs + 1 label → window size = seq_length+1
    window_size = seq_length + 1

    # slide a window of window_size, shift=1
    windows = dataset.window(window_size,
                             shift=1,
                             stride=1,
                             drop_remainder=True)

    # each `window` is a Dataset of Tensors, so batch them
    sequences = windows.flat_map(lambda w: w.batch(window_size))

    def split_and_format(seq):
        # seq: shape = (seq_length+1, 3)
        # inputs = first seq_length rows, label = last row
        inputs = seq[:-1]            # (seq_length, 3)
        label  = seq[-1]             # (3,)

        # unpack inputs into 3 separate tensors
        pitch_seq = tf.cast(inputs[:, 0], tf.int32)
        step_seq = tf.cast(inputs[:, 1], tf.int32)
        dur_seq  = tf.cast(inputs[:, 2], tf.int32)

        # format inputs as a dict matching your Input layer names
        x = {
            'p_in': pitch_seq,
            's_in': step_seq,
            'd_in': dur_seq
        }

        # unpack the label row into targets
        y = {
            'pitch':    tf.cast(label[0], tf.int32),       # integer class
            'step':     tf.cast(label[1], tf.int32),       # float in [0,1]
            'duration': tf.cast(label[2], tf.int32),       # float in [0,1]
        }

        return x, y

    return sequences.map(split_and_format,
                         num_parallel_calls=tf.data.AUTOTUNE)


Set the sequence length for each example. Experiment with different lengths (e.g. 50, 100, 150) to see which one works best for the data, or use [hyperparameter tuning](https://www.tensorflow.org/tutorials/keras/keras_tuner). The size of the vocabulary (`vocab_size`) is set to 128 representing all the pitches supported by `pretty_midi`.

In [ ]:
seq_length = 50
vocab_size = 128
seq_ds = create_sequences(notes_ds, seq_length, vocab_size)
seq_ds.element_spec

The shape of the dataset is ```(100,1)```, meaning that the model will take 100 notes as input, and learn to predict the following note as output.

In [ ]:
for seq, target in seq_ds.take(1):
    print("── Inputs ─────────────────────────────────")
    print("pitch sequence shape:   ", seq['p_in'].shape)
    print("step sequence shape:    ", seq['s_in'].shape)
    print("duration sequence shape:", seq['d_in'].shape)
    print()
    print("── A few pitch values ─────────────────────")
    print(seq['p_in'][0:10].numpy())     # first 10 pitch ints
    print()
    print("── Targets ────────────────────────────────")
    print("next pitch:",    target['pitch'].numpy())
    print("next step: ",    target['step'].numpy())
    print("next duration:", target['duration'].numpy())
    break


Batch the examples, and configure the dataset for performance.

In [48]:
batch_size = 64
buffer_size = n_notes - seq_length  # the number of items in the dataset
train_ds = (seq_ds
            .shuffle(buffer_size)
            .batch(batch_size, drop_remainder=True)
            .cache()
            .prefetch(tf.data.experimental.AUTOTUNE)
            .apply(tf.data.Dataset.ignore_errors))

In [ ]:
train_ds.element_spec

## Create and train the model

The model will have three outputs, one for each note variable. For `step` and `duration`, you will use a custom loss function based on mean squared error that encourages the model to output non-negative values.

In [50]:
def mse(y_true: tf.Tensor, y_pred: tf.Tensor):
  sq_err = (y_true - y_pred) ** 2
  return tf.reduce_mean(sq_err)
 

In [ ]:
learning_rate = 0.002
embed_dim=256

pitch_input = tf.keras.Input((seq_length,), dtype='int32', name='p_in')
step_input = tf.keras.Input((seq_length,), dtype='int32', name='s_in')
dur_input = tf.keras.Input((seq_length,), dtype='int32', name='d_in')
pitch_embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=False, name='pitch_embed')(pitch_input)
step_embed = tf.keras.layers.Embedding(input_dim=N, output_dim=embed_dim, mask_zero=False, name='step_embed')(step_input)
dur_embed = tf.keras.layers.Embedding(input_dim=N, output_dim=embed_dim, mask_zero=False, name='dur_embed')(dur_input)

x = tf.keras.layers.Concatenate(axis=-1)([pitch_embedding, step_embed, dur_embed])

x = tf.keras.layers.LSTM(512, return_sequences=True)(x)
x = tf.keras.layers.LSTM(512)(x)



# pitch stays as logits for your SparseCategoricalCrossentropy
pitch_logits = tf.keras.layers.Dense(128, name='pitch')(x)

# step and duration get a raw Dense(1) → softplus
step_raw     = tf.keras.layers.Dense(N, name="step")(x)
dur_raw      = tf.keras.layers.Dense(N, name="duration")(x)

model = tf.keras.Model({
    'p_in': pitch_input,
    's_in': step_input,
    'd_in': dur_input,
}, {
    'pitch': pitch_logits,
    'step': step_raw,
    'duration': dur_raw
})

loss = {
      'pitch': tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'step': tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True),
      'duration': tf.keras.losses.SparseCategoricalCrossentropy(
          from_logits=True)
}

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)

model.compile(loss=loss, optimizer=optimizer)

model.summary() 

Testing the `model.evaluate` function, you can see that the `pitch` loss is significantly greater than the `step` and `duration` losses. 
Note that `loss` is the total loss computed by summing all the other losses and is currently dominated by the `pitch` loss.

In [ ]:
losses = model.evaluate(train_ds, return_dict=True)
losses

One way balance this is to use the `loss_weights` argument to compile:

In [ ]:
avg = {'pitch': 4.85, 'step': 3.46, 'duration': 3.46}
inv   = { k: 1.0/v for k,v in avg.items() }
total = sum(inv.values())
weights = { k: inv[k]/total for k in inv }
print(weights)

model.compile(
    loss=loss,
    loss_weights=weights,
    optimizer=optimizer,
)

The `loss` then becomes the weighted sum of the individual losses.

In [ ]:
model.evaluate(train_ds, return_dict=True)

Train the model.

In [55]:
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='./training_checkpoints/ckpt_{epoch}.weights.h5',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=5,
        verbose=1,
        restore_best_weights=True),
]

In [ ]:
%%time

epochs = 50

history = model.fit(
    train_ds,
    epochs=epochs,
    callbacks=callbacks,
)

In [ ]:
plt.plot(history.epoch, history.history['loss'], label='total loss')
plt.show()

## Generate notes

To use the model to generate notes, you will first need to provide a starting sequence of notes. The function below generates one note from a sequence of notes. 

For note pitch, it draws a sample from the softmax distribution of notes produced by the model, and does not simply pick the note with the highest probability.
Always picking the note with the highest probability would lead to repetitive sequences of notes being generated.

The `temperature` parameter can be used to control the randomness of notes generated. You can find more details on temperature in [Text generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation).

In [58]:
def predict_next_note(
    notes: np.ndarray, 
    model: tf.keras.Model, 
    temperature: float = 1.0) -> tuple[int, float, float]:
  """Generates a note as a tuple of (pitch, step, duration), using a trained sequence model."""

  assert temperature > 0

  # 1) Unpack and cast the incoming notes array
  # ------------------------------------------------
  # notes[:, 0] are pitches (ints)
  # notes[:, 1] are steps (floats)
  # notes[:, 2] are durations (floats)
  seq_length = notes.shape[0]

  pitch_seq = notes[:, 0].astype('int32')             # (seq_length,)
  step_seq  = notes[:, 1].astype('int32')           # (seq_length,)
  dur_seq   = notes[:, 2].astype('int32')           # (seq_length,)

  # 2) Add the “batch” and “feature” dims as needed
  # ------------------------------------------------
  # pitch input is (batch, seq_length)
  p_in = np.expand_dims(pitch_seq, axis=0)            # (1, seq_length)

  # step/dur inputs are (batch, seq_length, 1)
  s_in = np.expand_dims(step_seq, axis=0)        # (1, seq_length)
  d_in = np.expand_dims(dur_seq,  axis=0)        # (1, seq_length)

  # 3) Predict
  # ------------------------------------------------
  predictions = model.predict({
      'p_in': p_in,
      's_in': s_in,
      'd_in': d_in
  }, verbose=0)
  pitch_logits = predictions['pitch']
  step_logits = predictions['step']
  duration_logits = predictions['duration']
 
  pitch_logits /= temperature
  pitch = tf.random.categorical(pitch_logits, num_samples=1)
  quantized_step = tf.random.categorical(step_logits, num_samples=1)
  quantized_duration = tf.random.categorical(duration_logits, num_samples=1)


  pitch = tf.squeeze(pitch, axis=-1)
  quantized_duration = tf.squeeze(quantized_duration, axis=-1)
  quantized_step = tf.squeeze(quantized_step, axis=-1)
  print(f"quantized_dur {quantized_duration} quantized_step {quantized_step}")

  duration = dequantize(np.array(quantized_duration), dur_edges)
  step = dequantize(np.array(quantized_step), step_edges)

  print(f"duration {duration} step {step}")
  
  return int(pitch), float(step), float(duration)

Now generate some notes. You can play around with temperature and the starting sequence in `next_notes` and see what happens.

In [ ]:
temperature = 1
num_predictions = 120

sample_notes = np.stack([all_notes[key] for key in key_order], axis=1)

# The initial sequence of notes; pitch is normalized similar to training
# sequences
input_notes = sample_notes[:seq_length]

df_notes = pd.DataFrame(
    all_notes[:seq_length], columns=(*key_order, 'start', 'end'))

df_notes["step"] = dequantize(df_notes["step"], step_edges)
df_notes["duration"] = dequantize(df_notes["duration"], dur_edges)

md_file = notes_to_midi(df_notes, out_file='output.mid', instrument_name=instrument_name)
display_audio(md_file)

generated_notes = []
prev_start = 0
for _ in range(num_predictions):
  pitch, step, duration = predict_next_note(input_notes, model, temperature)
  start = prev_start + step
  end = start + duration
  input_note = (pitch, step, duration)
  generated_notes.append((*input_note, start, end))
  input_notes = np.delete(input_notes, 0, axis=0)
  input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)
  prev_start = start

generated_notes = pd.DataFrame(
    generated_notes, columns=(*key_order, 'start', 'end'))

In [ ]:
generated_notes.head(50)

In [ ]:
out_file = 'output.mid'
out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name=instrument_name)
display_audio(out_pm)

In [ ]:
full_melody = pd.concat([df_notes, generated_notes])
out_pm = notes_to_midi(
    full_melody, out_file=out_file, instrument_name=instrument_name)
display_audio(out_pm)


You can also download the audio file by adding the two lines below:

```
from google.colab import files
files.download(out_file)
```

Visualize the generated notes.

In [ ]:
plot_piano_roll(generated_notes)

Check the distributions of `pitch`, `step` and `duration`.

In [ ]:
plot_distributions(generated_notes)

In the above plots, you will notice the change in distribution of the note variables.
Since there is a feedback loop between the model's outputs and inputs, the model tends to generate similar sequences of outputs to reduce the loss. 
This is particularly relevant for `step` and `duration`, which uses the MSE loss.
For `pitch`, you can increase the randomness by increasing the `temperature` in `predict_next_note`.


## Next steps

This tutorial demonstrated the mechanics of using an RNN to generate sequences of notes from a dataset of MIDI files. To learn more, you can visit the closely related [Text generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation) tutorial, which contains additional diagrams and explanations. 

One of the alternatives to using RNNs for music generation is using GANs. Rather than generating audio, a GAN-based approach can generate an entire sequence in parallel. The Magenta team has done impressive work on this approach with [GANSynth](https://magenta.tensorflow.org/gansynth). You can also find many wonderful music and art projects and open-source code on [Magenta project website](https://magenta.tensorflow.org/).